In [20]:
import pandas as pd
import os
from os import listdir
from inscriptis import get_text
import glob
from lxml import html
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
import pandas as pd
import random

In [16]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    temp = doc.replace('\n', ' ')
    temp = temp.replace('\xa0', ' ')
    temp = temp.replace('\x94', ' ')
    temp = temp.replace('\x92', ' ')
    temp = temp.replace('\x97', ' ')
    temp = temp.replace('\x93', ' ')
    temp = temp.replace('\x99', ' ')
    stop_free = " ".join([i for i in temp.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    no_num = ''.join([i for i in normalized if not i.isdigit()])
    singleletter_free = ' '.join( [w for w in no_num.split() if len(w)>1] )
    no_sth = ' '.join( [w for w in singleletter_free.split() if w != 'th'] )
    no_ic = ' '.join( [w for w in no_sth.split() if w != 'ic'] )

    return no_ic

### All Industries

Try with all texts within earnings call transcript

In [4]:
path = '/Users/sharontsao/Desktop/Datasets/EarningsCallCleaned'

In [5]:
industryFiles = [f for f in listdir(path) if '.csv' in f]

In [9]:
industryFiles

['EarningsCall_earningsCall_Biotechnology.csv',
 'EarningsCall_earningsCall_Semiconductors.csv',
 'EarningsCall_earningsCall_AerospaceDefense.csv',
 'EarningsCall_earningsCall_Computers.csv',
 'EarningsCall_earningsCall_CommercialServices.csv',
 'EarningsCall_earningsCall_Media.csv',
 'EarningsCall_earningsCall_Chemicals.csv',
 'earningsCall_Software.csv',
 'EarningsCall_earningsCall_Internet.csv',
 'EarningsCall_earningsCall_Apparel.csv']

In [30]:
All_content = []

for file in industryFiles:
    print(file)
    df = pd.read_csv('/Users/sharontsao/Desktop/Datasets/EarningsCallCleaned/' + file, encoding = 'ISO-8859-1')
    #df['Content']
    temp = df['Content'].dropna().tolist()
    random_elements = [random.choice(temp) for i in range(15000)]
    
    All_content.extend(random_elements)
    
    print(len(All_content))

EarningsCall_earningsCall_Biotechnology.csv
15000
EarningsCall_earningsCall_Semiconductors.csv
30000
EarningsCall_earningsCall_AerospaceDefense.csv
45000
EarningsCall_earningsCall_Computers.csv
60000
EarningsCall_earningsCall_CommercialServices.csv
75000
EarningsCall_earningsCall_Media.csv
90000
EarningsCall_earningsCall_Chemicals.csv
105000
earningsCall_Software.csv


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,8,10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


120000
EarningsCall_earningsCall_Internet.csv
135000
EarningsCall_earningsCall_Apparel.csv
150000


In [32]:
#cleanedList = [x for x in All_content if str(x) != 'nan']
doc_clean_all = [clean(doc).split() for doc in All_content]  

In [33]:
num_topic = 3

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=num_topic, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      46.390701        1       1 -0.132860 -0.126099
2      43.940334        1       2 -0.182362  0.113554
0       9.668974        1       3  0.315223  0.012545, topic_info=       Category           Freq        Term          Total  loglift  logprob
term                                                                       
1493    Default   70650.000000      ã¢ââ¢   70650.000000  30.0000  30.0000
25      Default   46572.000000    question   46572.000000  29.0000  29.0000
788     Default   66441.000000     current   66441.000000  28.0000  28.0000
139     Default   29998.000000       thank   29998.000000  27.0000  27.0000
268     Default   67615.000000       think   67615.000000  26.0000  26.0000
160     Default   30918.000000        next   30918.000000  25.0000  25.0000
151     Default   24960.000000         you   24960.000000  24.0000  24.0000
31      Default   28414.000000      thanks   28414.000000  23.0000  23.0000
77      Default   29582.000000   bloomberg   29582.000000  22.0000  22.0000
78      Default   39153.000000        call   39153.000000  21.0000  21.0000
762     Default   42789.000000     company   42789.000000  20.0000  20.0000
394     Default  113282.000000     quarter  113282.000000  19.0000  19.0000
849     Default   28635.000000    earnings   28635.000000  18.0000  18.0000
1110    Default   39726.000000     million   39726.000000  17.0000  17.0000
434     Default   27896.000000       great   27896.000000  16.0000  16.0000
158     Default   21571.000000        line   21571.000000  15.0000  15.0000
276     Default   18554.000000        okay   18554.000000  14.0000  14.0000
388     Default   30072.000000        name   30072.000000  13.0000  13.0000
154     Default   18189.000000        come   18189.000000  12.0000  12.0000
973     Default   33836.000000        good   33836.000000  11.0000  11.0000
282     Default   26017.000000    estimate   26017.000000  10.0000  10.0000
74      Default   34200.000000       ã¢ââ   34200.000000   9.0000   9.0000
1492    Default   24452.000000         ytd   24452.000000   8.0000   8.0000
255     Default   32696.000000      really   32696.000000   7.0000   7.0000
318     Default   35335.000000      change   35335.000000   6.0000   6.0000
1303    Default   51252.000000     revenue   51252.000000   5.0000   5.0000
201     Default   35624.000000       going   35624.000000   4.0000   4.0000
200     Default   18237.000000          go   18237.000000   3.0000   3.0000
265     Default   27237.000000       thats   27237.000000   2.0000   2.0000
142479  Default    8436.000000         û¢    8436.000000   1.0000   1.0000
...         ...            ...         ...            ...      ...      ...
91       Topic3    2531.837646   expressed    2900.640137   2.2003  -5.8419
141      Topic3    1914.851929  thirdparty    2112.878906   2.2378  -6.1213
25       Topic3   22034.390625    question   46572.304688   1.5878  -3.6783
31       Topic3   14526.853516      thanks   28414.341797   1.6654  -4.0949
154      Topic3    9386.030273        come   18189.058594   1.6747  -4.5317
276      Topic3    9320.803711        okay   18554.089844   1.6478  -4.5386
2499     Topic3    5673.793945     morning    9876.551758   1.7819  -5.0350
470      Topic3    4645.381836      please    7495.490234   1.8578  -5.2350
158      Topic3    9794.262695        line   21571.707031   1.5467  -4.4891
77       Topic3   11637.237305   bloomberg   29582.937500   1.4033  -4.3167
352      Topic3    4609.628418          hi    7807.285156   1.8093  -5.2427
2321     Topic3    4028.681152     michael    6807.268555   1.8117  -5.3775
1313     Topic3    1826.102539      robert    2086.163086   2.2031  -6.1687
90       Topic3    1892.167114     express    2217.711182   2.1775  -6.1332
434      Topic3    9263.920898       great   27896.890625   1.2339  -4.5448
302      Topic3    3999.328613        o

In [34]:
num_topic = 4

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=num_topic, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      44.159088        1       1  0.065101 -0.143868
2      30.925470        1       2  0.183775 -0.028674
1      19.456146        1       3  0.110026  0.163269
0       5.459304        1       4 -0.358902  0.009274, topic_info=      Category           Freq       Term          Total  loglift  logprob
term                                                                     
1493   Default   58390.000000     ã¢ââ¢   58390.000000  30.0000  30.0000
788    Default   66027.000000    current   66027.000000  29.0000  29.0000
25     Default   43479.000000   question   43479.000000  28.0000  28.0000
160    Default   28175.000000       next   28175.000000  27.0000  27.0000
139    Default   26169.000000      thank   26169.000000  26.0000  26.0000
762    Default   42303.000000    company   42303.000000  25.0000  25.0000
394    Default  115474.000000    quarter  115474.000000  24.0000  24.0000
78     Default   38470.000000       call   38470.000000  23.0000  23.0000
151    Default   22234.000000        you   22234.000000  22.0000  22.0000
849    Default   28096.000000   earnings   28096.000000  21.0000  21.0000
268    Default   67398.000000      think   67398.000000  20.0000  20.0000
31     Default   26323.000000     thanks   26323.000000  19.0000  19.0000
71     Default   53388.000000       were   53388.000000  18.0000  18.0000
282    Default   25837.000000   estimate   25837.000000  17.0000  17.0000
158    Default   20357.000000       line   20357.000000  16.0000  16.0000
1492   Default   24220.000000        ytd   24220.000000  15.0000  15.0000
388    Default   29051.000000       name   29051.000000  14.0000  14.0000
77     Default   28282.000000  bloomberg   28282.000000  13.0000  13.0000
154    Default   16842.000000       come   16842.000000  12.0000  12.0000
434    Default   26634.000000      great   26634.000000  11.0000  11.0000
276    Default   17190.000000       okay   17190.000000  10.0000  10.0000
318    Default   35203.000000     change   35203.000000   9.0000   9.0000
38     Default  115766.000000       year  115766.000000   8.0000   8.0000
200    Default   17362.000000         go   17362.000000   7.0000   7.0000
255    Default   32553.000000     really   32553.000000   6.0000   6.0000
1303   Default   52941.000000    revenue   52941.000000   5.0000   5.0000
74     Default   34169.000000      ã¢ââ   34169.000000   4.0000   4.0000
1087   Default   34171.000000     margin   34171.000000   3.0000   3.0000
17     Default   19044.000000     little   19044.000000   2.0000   2.0000
201    Default   35585.000000      going   35585.000000   1.0000   1.0000
...        ...            ...        ...            ...      ...      ...
2751    Topic4     810.956787    proceed     882.987915   2.8228  -6.4091
151     Topic4   12169.663086        you   22234.087891   2.3052  -3.7006
1493    Topic4   24400.048828     ã¢ââ¢   58390.984375   2.0353  -3.0050
25      Topic4   17237.953125   question   43479.214844   1.9827  -3.3525
154     Topic4    8079.671387       come   16842.863281   2.1733  -4.1102
470     Topic4    4030.024902     please    6916.087402   2.3678  -4.8058
2321    Topic4    3709.764648    michael    6297.489746   2.3787  -4.8886
20347   Topic4    1892.983643       omar    2703.670654   2.5514  -5.5614
158     Topic4    8058.150879       line   20357.251953   1.9811  -4.1129
31      Topic4    8978.273438     thanks   26323.402344   1.8322  -4.0048
276     Topic4    6626.089355       okay   17190.927734   1.9545  -4.3086
302     Topic4    3492.298584       open    7170.340820   2.1885  -4.9490
200     Topic4    5230.584961         go   17362.298828   1.7081  -4.5451
793     Topic4    1807.206787      david    3267.257812   2.3157  -5.6078
849     Topic4    6610.112793   earnings   28096.070312   1.4608  -4.3110
788     Topic4   10766.089844    current   66027.507812   1.0942  -3.8232
434    

In [35]:
num_topic = 6

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=num_topic, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      40.870926        1       1  0.150791 -0.129841
5      30.329504        1       2  0.230195  0.026782
1      12.685980        1       3  0.170500  0.135858
2       7.928225        1       4 -0.009239 -0.139361
3       4.566901        1       5 -0.309451 -0.179632
4       3.618463        1       6 -0.232797  0.286194, topic_info=      Category           Freq       Term          Total  loglift  logprob
term                                                                     
1493   Default   56795.000000     ã¢ââ¢   56795.000000  30.0000  30.0000
788    Default   65283.000000    current   65283.000000  29.0000  29.0000
25     Default   45032.000000   question   45032.000000  28.0000  28.0000
394    Default  116702.000000    quarter  116702.000000  27.0000  27.0000
160    Default   28007.000000       next   28007.000000  26.0000  26.0000
762    Default   41705.000000    company   41705.000000  25.0000  25.0000
139    Default   26444.000000      thank   26444.000000  24.0000  24.0000
38     Default  115656.000000       year  115656.000000  23.0000  23.0000
849    Default   27676.000000   earnings   27676.000000  22.0000  22.0000
151    Default   22540.000000        you   22540.000000  21.0000  21.0000
78     Default   38272.000000       call   38272.000000  20.0000  20.0000
31     Default   27413.000000     thanks   27413.000000  19.0000  19.0000
318    Default   34927.000000     change   34927.000000  18.0000  18.0000
268    Default   65863.000000      think   65863.000000  17.0000  17.0000
282    Default   25544.000000   estimate   25544.000000  16.0000  16.0000
158    Default   20280.000000       line   20280.000000  15.0000  15.0000
1492   Default   23934.000000        ytd   23934.000000  14.0000  14.0000
77     Default   28400.000000  bloomberg   28400.000000  13.0000  13.0000
434    Default   27144.000000      great   27144.000000  12.0000  12.0000
154    Default   16523.000000       come   16523.000000  11.0000  11.0000
70     Default   44598.000000       well   44598.000000  10.0000  10.0000
276    Default   17924.000000       okay   17924.000000   9.0000   9.0000
71     Default   51563.000000       were   51563.000000   8.0000   8.0000
200    Default   16992.000000         go   16992.000000   7.0000   7.0000
255    Default   31229.000000     really   31229.000000   6.0000   6.0000
881    Default   17088.000000        eps   17088.000000   5.0000   5.0000
388    Default   29011.000000       name   29011.000000   4.0000   4.0000
133    Default   39907.000000       sale   39907.000000   3.0000   3.0000
17     Default   20057.000000     little   20057.000000   2.0000   2.0000
1087   Default   35091.000000     margin   35091.000000   1.0000   1.0000
...        ...            ...        ...            ...      ...      ...
154     Topic6    9742.501953       come   16523.443359   2.7908  -3.5119
25      Topic6   21151.556641   question   45032.687500   2.5634  -2.7367
302     Topic6    4452.804688       open    7161.707031   2.8439  -4.2949
20347   Topic6    1930.779785       omar    2668.901123   2.9954  -5.1305
158     Topic6    9966.613281       line   20280.316406   2.6087  -3.4892
470     Topic6    3929.909424     please    6843.944336   2.7644  -4.4198
42465   Topic6     778.136108      wayne     915.364075   3.1567  -6.0393
276     Topic6    8138.208008       okay   17924.025391   2.5295  -3.6918
15498   Topic6    2524.675049   retailer    4140.637207   2.8244  -4.8623
2787    Topic6    1185.238647       bank    1571.507568   3.0370  -5.6185
31      Topic6    9707.997070     thanks   27413.468750   2.2810  -3.5155
200     Topic6    6258.950195         go   16992.419922   2.3204  -3.9544
434     Topic6    7858.917480      great   27144.412109   2.0796  -3.7268
451     Topic6    2632.703125      ahead    7031.854004   2.3367  -4.8204
62      Topic6    2956.614258       take   1533

In [36]:
num_topic = 8

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=num_topic, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      30.984562        1       1 -0.125113  0.170222
4      24.924177        1       2 -0.141906 -0.102392
3      16.826582        1       3 -0.212512  0.091610
5       9.478878        1       4 -0.176932 -0.088548
1       5.550953        1       5  0.211588 -0.221183
6       5.307587        1       6 -0.051918 -0.062613
7       4.669305        1       7  0.251502  0.304879
0       2.257951        1       8  0.245290 -0.091976, topic_info=     Category           Freq            Term          Total  loglift  logprob
term                                                                         
1493  Default   54399.000000          ã¢ââ¢   54399.000000  30.0000  30.0000
788   Default   68653.000000         current   68653.000000  29.0000  29.0000
25    Default   43634.000000        question   43634.000000  28.0000  28.0000
268   Default   71512.000000           think   71512.000000  27.0000  27.0000
394   Default  115634.000000         quarter  115634.000000  26.0000  26.0000
719   Default   39021.000000           brand   39021.000000  25.0000  25.0000
762   Default   42898.000000         company   42898.000000  24.0000  24.0000
38    Default  116572.000000            year  116572.000000  23.0000  23.0000
31    Default   26320.000000          thanks   26320.000000  22.0000  22.0000
78    Default   39908.000000            call   39908.000000  21.0000  21.0000
77    Default   28918.000000       bloomberg   28918.000000  20.0000  20.0000
160   Default   28040.000000            next   28040.000000  19.0000  19.0000
139   Default   25967.000000           thank   25967.000000  18.0000  18.0000
849   Default   28989.000000        earnings   28989.000000  17.0000  17.0000
202   Default   64562.000000          growth   64562.000000  16.0000  16.0000
318   Default   36329.000000          change   36329.000000  15.0000  15.0000
282   Default   26844.000000        estimate   26844.000000  14.0000  14.0000
151   Default   22067.000000             you   22067.000000  13.0000  13.0000
1492  Default   25067.000000             ytd   25067.000000  12.0000  12.0000
1110  Default   45333.000000         million   45333.000000  11.0000  11.0000
71    Default   53864.000000            were   53864.000000  10.0000  10.0000
347   Default   73027.000000        business   73027.000000   9.0000   9.0000
132   Default   21439.000000           right   21439.000000   8.0000   8.0000
973   Default   32971.000000            good   32971.000000   7.0000   7.0000
133   Default   39883.000000            sale   39883.000000   6.0000   6.0000
434   Default   24975.000000           great   24975.000000   5.0000   5.0000
276   Default   17108.000000            okay   17108.000000   4.0000   4.0000
74    Default   35708.000000           ã¢ââ   35708.000000   3.0000   3.0000
1303  Default   54988.000000         revenue   54988.000000   2.0000   2.0000
201   Default   36997.000000           going   36997.000000   1.0000   1.0000
...       ...            ...             ...            ...      ...      ...
129    Topic8     933.070496        reserved     933.859070   3.7899  -5.3858
134    Topic8     930.939148    securitiesor     931.727112   3.7899  -5.3881
131    Topic8     911.765686  retransmission     912.553772   3.7898  -5.4089
104    Topic8    1538.500000            lost    1598.823975   3.7523  -4.8857
141    Topic8    1643.991211      thirdparty    1729.464233   3.7400  -4.8194
91     Topic8    2196.496094       expressed    2406.000000   3.6996  -4.5296
76     Topic8    1199.669556           agent    1234.572876   3.7620  -5.1344
138    Topic8    1734.405396        supplier    1877.758545   3.7113  -4.7658
94     Topic8    1585.800903         implied    1763.270264   3.6846  -4.8554
116    Topic8    2134.747070         percent    2599.408447   3.5938  -4.5581
118    Topic8    1396.596191        personal    1719.218750   3

In [37]:
num_topic = 10

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=num_topic, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      31.056473        1       1 -0.218201 -0.055422
0      25.726006        1       2 -0.175940 -0.056257
2      10.257773        1       3 -0.199142  0.154990
3       8.954299        1       4 -0.197536 -0.070939
4       6.924628        1       5 -0.110762  0.145847
5       4.756495        1       6  0.112666 -0.220688
6       3.970458        1       7  0.126082 -0.139120
8       3.367925        1       8  0.189761  0.224593
9       3.073586        1       9  0.219895 -0.195238
1       1.912350        1      10  0.253177  0.212235, topic_info=       Category           Freq           Term          Total  loglift  logprob
term                                                                          
1493    Default   54246.000000         ã¢ââ¢   54246.000000  30.0000  30.0000
788     Default   68656.000000        current   68656.000000  29.0000  29.0000
394     Default  114226.000000        quarter  114226.000000  28.0000  28.0000
38      Default  114886.000000           year  114886.000000  27.0000  27.0000
25      Default   42517.000000       question   42517.000000  26.0000  26.0000
268     Default   71098.000000          think   71098.000000  25.0000  25.0000
160     Default   27939.000000           next   27939.000000  24.0000  24.0000
762     Default   42356.000000        company   42356.000000  23.0000  23.0000
133160  Default   20594.000000       footwear   20594.000000  22.0000  22.0000
139     Default   25800.000000          thank   25800.000000  21.0000  21.0000
78      Default   39932.000000           call   39932.000000  20.0000  20.0000
202     Default   65598.000000         growth   65598.000000  19.0000  19.0000
347     Default   76795.000000       business   76795.000000  18.0000  18.0000
849     Default   28982.000000       earnings   28982.000000  17.0000  17.0000
31      Default   25440.000000         thanks   25440.000000  16.0000  16.0000
318     Default   36447.000000         change   36447.000000  15.0000  15.0000
973     Default   33408.000000           good   33408.000000  14.0000  14.0000
151     Default   21947.000000            you   21947.000000  13.0000  13.0000
282     Default   26840.000000       estimate   26840.000000  12.0000  12.0000
719     Default   29240.000000          brand   29240.000000  11.0000  11.0000
98199   Default   17158.000000        apparel   17158.000000  10.0000  10.0000
1492    Default   25087.000000            ytd   25087.000000   9.0000   9.0000
71      Default   52971.000000           were   52971.000000   8.0000   8.0000
77      Default   28890.000000      bloomberg   28890.000000   7.0000   7.0000
1087    Default   30810.000000         margin   30810.000000   6.0000   6.0000
158     Default   19970.000000           line   19970.000000   5.0000   5.0000
9805    Default   22453.000000          store   22453.000000   4.0000   4.0000
154     Default   16942.000000           come   16942.000000   3.0000   3.0000
200     Default   17991.000000             go   17991.000000   2.0000   2.0000
212     Default   37488.000000        product   37488.000000   1.0000   1.0000
...         ...            ...            ...            ...      ...      ...
41464   Topic10     497.841003           ruth     498.685394   3.9551  -5.8477
29858   Topic10     496.248047       vacation     497.092316   3.9551  -5.8509
98075   Topic10     475.388214            cpc     476.233307   3.9551  -5.8939
12867   Topic10     459.531921            rob     460.376282   3.9550  -5.9278
13446   Topic10     449.071472          fresh     449.916138   3.9550  -5.9508
8910    Topic10     446.761536         module     447.605713   3.9549  -5.9560
3735    Topic10     370.666290        carrier     371.510345   3.9546  -6.1427
15087   Topic10     369.776855         opaque     370.621429   3.9546  -6.1451
98862   Topic10     339.603149           cpcs     340.447937   3.9544 

In [38]:
num_topic = 12

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=num_topic, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      31.231579        1       1 -0.155209 -0.084269
10     16.616478        1       2 -0.211996 -0.039790
4      12.244323        1       3 -0.230204 -0.033647
11      9.668931        1       4 -0.174749 -0.111721
8       6.597569        1       5 -0.085191  0.240943
3       6.570351        1       6 -0.154984  0.184282
1       3.951544        1       7  0.193312 -0.176913
2       3.779962        1       8  0.139743  0.188653
7       3.463390        1       9  0.158690 -0.132850
0       2.787334        1      10  0.073114 -0.165688
6       1.713745        1      11  0.177833 -0.065497
5       1.374797        1      12  0.269641  0.196497, topic_info=       Category           Freq           Term          Total  loglift  logprob
term                                                                          
394     Default  115531.000000        quarter  115531.000000  30.0000  30.0000
1493    Default   55962.000000         ã¢ââ¢   55962.000000  29.0000  29.0000
788     Default   66916.000000        current   66916.000000  28.0000  28.0000
25      Default   44799.000000       question   44799.000000  27.0000  27.0000
38      Default  114012.000000           year  114012.000000  26.0000  26.0000
973     Default   31632.000000           good   31632.000000  25.0000  25.0000
268     Default   70164.000000          think   70164.000000  24.0000  24.0000
160     Default   28067.000000           next   28067.000000  23.0000  23.0000
139     Default   25855.000000          thank   25855.000000  22.0000  22.0000
71      Default   54564.000000           were   54564.000000  21.0000  21.0000
719     Default   26605.000000          brand   26605.000000  20.0000  20.0000
77      Default   28171.000000      bloomberg   28171.000000  19.0000  19.0000
151     Default   22111.000000            you   22111.000000  18.0000  18.0000
762     Default   42080.000000        company   42080.000000  17.0000  17.0000
78      Default   38981.000000           call   38981.000000  16.0000  16.0000
433     Default   34912.000000          first   34912.000000  15.0000  15.0000
849     Default   28310.000000       earnings   28310.000000  14.0000  14.0000
282     Default   26176.000000       estimate   26176.000000  13.0000  13.0000
318     Default   35883.000000         change   35883.000000  12.0000  12.0000
1110    Default   42630.000000        million   42630.000000  11.0000  11.0000
347     Default   70252.000000       business   70252.000000  10.0000  10.0000
1492    Default   24499.000000            ytd   24499.000000   9.0000   9.0000
31      Default   26906.000000         thanks   26906.000000   8.0000   8.0000
202     Default   64167.000000         growth   64167.000000   7.0000   7.0000
212     Default   40303.000000        product   40303.000000   6.0000   6.0000
158     Default   19346.000000           line   19346.000000   5.0000   5.0000
1325    Default   20751.000000         second   20751.000000   4.0000   4.0000
276     Default   17764.000000           okay   17764.000000   3.0000   3.0000
154     Default   16305.000000           come   16305.000000   2.0000   2.0000
133     Default   39144.000000           sale   39144.000000   1.0000   1.0000
...         ...            ...            ...            ...      ...      ...
141975  Topic12     736.977661          plank     737.892761   4.2856  -5.1255
22041   Topic12     705.586304         rental     706.501221   4.2856  -5.1691
175682  Topic12     690.454834      outerwear     691.370422   4.2855  -5.1907
176209  Topic12     646.681335    shopinshops     647.596436   4.2854  -5.2562
3273    Topic12     587.611267   domestically     588.526184   4.2853  -5.3520
175999  Topic12     552.686523        handbag     553.601990   4.2852  -5.4133
175954  Topic12     540.492798        leather     541.407837   4.2852  -5.4356
2636    Topic12     531.817871   deprecia